<a href="https://colab.research.google.com/github/ucaokylong/LLM_learning/blob/main/Text_embedding_advanced_for_task_legal_document_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

U Cao Ky Long

#Huấn luyện mô hình embedding cho task document Retrieval trên bộ dữ liệu BKAI Legal Document Retrieval với thư viện sentence-transformer

## Cài đặt thư viện

In [1]:
!pip install -q transformers==4.45.2 sentence-transformers==3.1.1
!pip install -q mteb wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207

## Import thư viện và các hyperparameters

In [2]:
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, losses, models
from sentence_transformers.readers import InputExample
import os
import pandas as pd
from ast import literal_eval

is_word = False
train_batch_size = 32
num_epochs = 10
model_save_path = os.path.join(os.getcwd(), 'output', 'model')

/usr/local/lib/python3.11/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


## Tải về bộ dữ liệu BKAI Legal Document Retrieval

In [4]:
!wget https://huggingface.co/datasets/tmnam20/BKAI-Legal-Retrieval/resolve/main/archive.zip
!unzip -o -qq archive.zip -d data

--2025-02-22 18:14:27--  https://huggingface.co/datasets/tmnam20/BKAI-Legal-Retrieval/resolve/main/archive.zip
Resolving huggingface.co (huggingface.co)... 18.164.174.118, 18.164.174.55, 18.164.174.17, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.118|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/77/34/7734ba2101c276edd0fc8c24cc368330a94a50e26dadb6b459834f0603785f20/35bef18231742a2ec66889a9074a8283b0807448b777bf8dda45a95859d0e875?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27archive.zip%3B+filename%3D%22archive.zip%22%3B&response-content-type=application%2Fzip&Expires=1740251667&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MDI1MTY2N319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzc3LzM0Lzc3MzRiYTIxMDFjMjc2ZWRkMGZjOGMyNGNjMzY4MzMwYTk0YTUwZTI2ZGFkYjZiNDU5ODM0ZjA2MDM3ODVmMjAvMzViZWYxODIzMTc0MmEyZWM2Njg4OWE5MDc0YTgyODNiMDgwNzQ0OGI3Nz

Sau khi giải nén, ta có các file như sau trong folder data:
- corpus.csv
- public.csv
- readme.txt
- train_split.csv
- train.csv
- val_split.csv

Trong đó file corpus chứa các đoạn văn luật mà mình cần tìm kiếm. File train_split.csv và val_split.csv chứa dữ liệu huấn luyện. Mỗi sample gồm query, context, context id và query id.

## Tiền xử lý

In [5]:
corpus_data = pd.read_csv('data/corpus.csv')
train_data = pd.read_csv('data/train_split.csv', converters={'context': literal_eval})
test_data = pd.read_csv('data/val_split.csv', converters={'context': literal_eval})

In [6]:
train_data['cid'] = train_data['cid'].apply(lambda x: [int(i) for i in x[1:-1].split()])
test_data['cid'] = test_data['cid'].apply(lambda x: [int(i) for i in x[1:-1].split()])

In [7]:
samples = {'train': [], 'test': []}
data = {'train': train_data, 'test': test_data}

for subset in ['train', 'test']:
    for i, row in data[subset].iterrows():
        question = row['question']
        context = row['context']
        for c in context:
            samples[subset].append(InputExample(texts=[question, c]))

train_dataloader = DataLoader(samples['train'], shuffle=True, batch_size=train_batch_size)

## Load pre-trained model

In [8]:
model_id = "google-bert/bert-base-multilingual-cased"
word_embedding_model = models.Transformer(model_id, max_seq_length=512, cache_dir='./cache')
pooling_model = models.Pooling(
    word_embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=False,
    pooling_mode_cls_token=True,
    pooling_mode_max_tokens=False,
)
model = SentenceTransformer(modules=[word_embedding_model, pooling_model], device='cuda', cache_folder='./cache')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

## Train the embedding model

In [ ]:
train_loss = losses.CachedMultipleNegativesRankingLoss(model=model)

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=num_epochs,
    steps_per_epoch=num_epochs,
    output_path=model_save_path,
    optimizer_params={'lr': 3e-5},
)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ucaokylong-hardcore (ucaokylong-hardcore-ai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
500,1.787800


## Đánh giá mô hình

In [ ]:
from mteb import MTEB
from mteb.abstasks.AbsTaskRetrieval import AbsTaskRetrieval
from mteb.abstasks.TaskMetadata import TaskMetadata

class BKAILegalDocRetrievalTask(AbsTaskRetrieval):
    metadata = TaskMetadata(
        name="BKAILegalDocRetrieval",
        description="",
        reference="https://github.com/embeddings-benchmark/mteb/blob/main/docs/adding_a_dataset.md",
        type="Retrieval",
        category="s2p",
        modalities=["text"],
        eval_splits=["test"],
        eval_langs=["vi"],
        main_score="ndcg_at_10",
        dataset={
            "path": "data",
            "revision": "d4c5a8ba10ae71224752c727094ac4c46947fa29",
        },
        date=("2012-01-01", "2020-01-01"),
        form="Written",
        domains=["Academic", "Non-fiction"],
        task_subtypes=["Scientific Reranking"],
        license="cc-by-nc-4.0",
        annotations_creators="derived",
        dialect=[],
        text_creation="found",
        bibtex_citation="",
    )
    data_loaded = True

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        global corpus_data, data
        self.corpus = {}
        self.queries = {}
        self.relevant_docs = {}
        shared_corpus = {}
        for _, row in corpus_data.iterrows():
            cid_str = f'c{row["cid"]}'
            shared_corpus[cid_str] = {"text": row['text'], "id": row['cid']}
        for split in data:
            self.corpus[split] = shared_corpus
            self.queries[split] = {}
            self.relevant_docs[split] = {}
        for split in data:
            for i, row in data[split].iterrows():
                qid, cids = row['qid'], row['cid']
                question = row['question']
                qid_str, cids_str = f'q{qid}', [f'c{cid}' for cid in cids]
                self.queries[split][qid_str] = question
                for cid_str in cids_str:
                    if cid_str not in self.relevant_docs[split]:
                        self.relevant_docs[split][qid_str] = {}
                    self.relevant_docs[split][qid_str][cid_str] = 1
        self.data_loaded = True

In [ ]:
custom_task = BKAILegalDocRetrievalTask()
evaluation = MTEB(tasks=[custom_task])
evaluation.run(model)

## Truy vấn

In [ ]:
passages = corpus_data['text'].tolist()
print(f'Number of passages: {len(passages)}')

corpus_embeddings = model.encode(passages, convert_to_tensor=True, show_progress_bar=True)

def search(query, top_k=5):
    print("Input question:", query)
    question_embedding = model.encode(query, convert_to_tensor=True)
    question_embedding = question_embedding.cuda()
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    hits = hits[0]  # Get the hits for the first query
    print("\n---\n")
    print("Top-3 Bi-Encoder Retrieval hits")
    hits = sorted(hits, key=lambda x: x['score'], reverse=True)
    for hit in hits[0:3]:
        print("\t{:.3f}\t{}".format(hit['score'], passages[hit['corpus_id']].replace("\n", " ")))
    return hits


In [ ]:
# Example usage
search(query="Lực lượng tuần tra, canh gác được trang bị những gì")